In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
file_path = r"D:\PROJECT\sites\us_kon\us_kon.csv"
df = pd.read_csv(file_path)
df.replace(-9990, np.nan, inplace=True)
wet_year = 2008
drought_year = 2012
normal_year = 2015
year_colors = {
    wet_year: 'darkblue',
    drought_year: 'darkred',
    normal_year: 'darkgreen'
}


In [ ]:
# ET CALC
latent_heat_of_vaporization = 2.45*10**6  # MJ/kg
df['ET_eddy_cov'] = df['LE']*1000 / latent_heat_of_vaporization
df.to_csv(file_path , index=False)
print("New column named ET_eddy_cov is created in the new CSV file.")

In [ ]:
df = pd.read_csv("D:\\PROJECT\\sites\\DBF\\MISSOURI\\MISSOURI.csv",  parse_dates=['TIMESTAMP_START'])
df['TIMESTAMP_START'] = pd.to_datetime(df['TIMESTAMP_START'], format='%d-%m-%Y %H:%M', errors='coerce')
df['Year'] = df['TIMESTAMP_START'].dt.year
df['Month'] = df['TIMESTAMP_START'].dt.month
df['Hour'] = df['TIMESTAMP_START'].dt.hour
df['Minute'] = df['TIMESTAMP_START'].dt.minute
time_slots = [(0, 0), (0, 30), (1, 0), (1, 30), (2, 0), (2, 30), (3, 0), (3, 30), (4, 0), (4, 30),
               (5, 0), (5, 30), (6, 0), (6, 30), (7, 0), (7, 30), (8, 0), (8, 30), (9, 0), (9, 30),
               (10, 0), (10, 30), (11, 0), (11, 30), (12, 0), (12, 30), (13, 0), (13, 30), (14, 0), (14, 30),
               (15, 0), (15, 30), (16, 0), (16, 30), (17, 0), (17, 30), (18, 0), (18, 30), (19, 0), (19, 30),
               (20, 0), (20, 30), (21, 0), (21, 30), (22, 0), (22, 30), (23, 0), (23, 30)]


df_filtered = df[df[['Hour', 'Minute']].apply(tuple, axis=1).isin(time_slots)]

average_temp = df_filtered.groupby(['Year', 'Month','Hour', 'Minute'])['TEMP'].mean().reset_index()
average_VPD = df_filtered.groupby(['Year', 'Month','Hour', 'Minute'])['VPD'].mean().reset_index()
#average_Rn = df_filtered.groupby(['Year', 'Month','Hour', 'Minute'])['Net_Radiation'].mean().reset_index()
average_ET = df_filtered.groupby(['Year', 'Month','Hour', 'Minute'])['ET_eddy_cov'].mean().reset_index()
average_GPP_NT = df_filtered.groupby(['Year', 'Month','Hour', 'Minute'])['GPP_NT_VUT_REF'].mean().reset_index()
average_GPP_DT = df_filtered.groupby(['Year', 'Month','Hour', 'Minute'])['GPP_DT_VUT_REF'].mean().reset_index()

# Rename columns for clarity
average_temp.columns = ['Year', 'Month','Hour', 'Minute', 'Average_Temperature']
average_VPD.columns = ['Year', 'Month','Hour', 'Minute', 'Average_VPD']
#average_Rn.columns = ['Year', 'Month','Hour', 'Minute', 'Average_net_radiation']
average_ET.columns = ['Year', 'Month','Hour', 'Minute', 'Average_ET']
average_GPP_NT.columns = ['Year', 'Month','Hour', 'Minute', 'Average_GPP_NT']
average_GPP_DT.columns = ['Year', 'Month','Hour', 'Minute', 'Average_GPP_DT']

average_ET = average_ET[average_ET['Average_ET'] >= 0]
average_GPP_NT = average_GPP_NT[average_GPP_NT['Average_GPP_NT'] >= 0]
average_GPP_DT = average_GPP_DT[average_GPP_DT['Average_GPP_DT'] >= 0]

## YEARLY PLOTS

In [ ]:
import matplotlib.pyplot as plt
merged_data = pd.merge(average_GPP_DT, average_ET, on=['Year','Month','Hour','Minute'])
variables=[('Average_GPP_DT','Average_ET','GPP vs ET')]
month_names=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
years=merged_data['Year'].unique()
for year in years:
    plt.figure(figsize=(15,10))
    plt.suptitle(f'Year: {year}',fontsize=16)
    for month in range(1,13):
        ax=plt.subplot(3,4,month)
        monthly_data=merged_data[(merged_data['Year']==year)&(merged_data['Month']==month)]
        sorted_data=monthly_data.sort_values(by=['Hour','Minute'])
        ax.scatter(sorted_data['Average_GPP_DT'],sorted_data['Average_ET'],alpha=0.5,label='GPP vs ET',edgecolor='none')
        for j in range(len(sorted_data)-1):
            x_start=sorted_data.iloc[j]['Average_GPP_DT']
            y_start=sorted_data.iloc[j]['Average_ET']
            x_end=sorted_data.iloc[j+1]['Average_GPP_DT']
            y_end=sorted_data.iloc[j+1]['Average_ET']
            ax.annotate('',xy=(x_end,y_end),xytext=(x_start,y_start),arrowprops=dict(arrowstyle='->',color='grey',lw=1.0))
        ax.set_title(f'{month_names[month-1]}')
        ax.set_xlabel('Average_GPP_DT')
        ax.set_ylabel('Average_ET')
    plt.tight_layout(rect=[0,0,1,0.96])
    plt.show()


## MONTHLY PLOTS

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
merged_data=pd.merge(average_GPP_DT,average_ET,on=['Year','Month','Hour','Minute'])
variables=[('Average_GPP_DT','Average_ET','GPP vs ET')]
month_names=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
years=merged_data['Year'].unique()
for month in range(1,13):
    plt.figure(figsize=(15,10))
    rows=(len(years)+3)//4
    cols=4
    for i,year in enumerate(years):
        ax=plt.subplot(rows,cols,i+1)
        monthly_data=merged_data[(merged_data['Year']==year)&(merged_data['Month']==month)]
        sorted_data=monthly_data.sort_values(by=['Hour','Minute'])
        ax.scatter(sorted_data['Average_GPP_DT'],sorted_data['Average_ET'],alpha=0.5,label=f'Year {year}',edgecolor='none')
        for j in range(len(sorted_data)-1):
            x_start=sorted_data.iloc[j]['Average_GPP_DT']
            y_start=sorted_data.iloc[j]['Average_ET']
            x_end=sorted_data.iloc[j+1]['Average_GPP_DT']
            y_end=sorted_data.iloc[j+1]['Average_ET']
            ax.annotate('',xy=(x_end,y_end),xytext=(x_start,y_start),arrowprops=dict(arrowstyle='->',color='grey',lw=1.0))
        ax.set_title(f'{month_names[month-1]} - {year}')
        ax.set_xlabel('Average_GPP_DT')
        ax.set_ylabel('Average_ET')
    plt.tight_layout(rect=[0,0,1,0.96])
    plt.suptitle(f'GPP vs ET for {month_names[month-1]}',fontsize=16)
    plt.show()


## DROUGHT 
Identify and enter the wet yr(darkblue), drought yr(dark red) and normal yr(dark green)

In [ ]:
merged_data = merged_data[merged_data['Year'].isin([wet_year, drought_year, normal_year])]
merged_data = merged_data[merged_data['Month'].between(4, 9)]
month_names = ['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
year_colors = {wet_year: 'blue', drought_year: 'red', normal_year: 'green'}
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Normalized GPP DT vs Normalized ET (April to September)', fontsize=20)
for i, month in enumerate(range(4, 10)):
    ax = axes[i // 3, i % 3]
    monthly_data = merged_data[merged_data['Month'] == month]
    GPP_max = monthly_data['Average_GPP_DT'].max()
    ET_max = monthly_data['Average_ET'].max()
    monthly_data['Normalized_GPP'] = monthly_data['Average_GPP_DT'] / GPP_max
    monthly_data['Normalized_ET'] = monthly_data['Average_ET'] / ET_max
    for year, color in year_colors.items():
        year_data = monthly_data[monthly_data['Year'] == year]
        sorted_data = year_data.sort_values(by=['Hour', 'Minute'])
        ax.scatter(sorted_data['Normalized_ET'], sorted_data['Normalized_GPP'], alpha=0.5, label=str(year), edgecolor='none', color=color)
        for j in range(len(sorted_data) - 1):
            x_start = sorted_data.iloc[j]['Normalized_ET']
            y_start = sorted_data.iloc[j]['Normalized_GPP']
            x_end = sorted_data.iloc[j + 1]['Normalized_ET']
            y_end = sorted_data.iloc[j + 1]['Normalized_GPP']
            ax.annotate('', xy=(x_end, y_end), xytext=(x_start, y_start), arrowprops=dict(arrowstyle='->', color='grey', lw=1.5))
    ax.set_title(month_names[i])
    ax.set_xlabel('ET/ET max', fontsize=16)
    ax.set_ylabel('GPP/GPPmax', fontsize=16)
    ax.legend(title='Year')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
merged_data=merged_data[merged_data['Year'].isin([wet_year,drought_year,normal_year])]
seasons={'Spring':[3,4,5],'Summer':[6,7,8],'Fall':[9,10,11]}
season_colors={'Spring':'green','Summer':'orange','Fall':'brown'}
year_colors={wet_year:'blue',drought_year:'red',normal_year:'green'}
areas={'Spring':{},'Summer':{},'Fall':{}}
fig,axes=plt.subplots(1,3,figsize=(24,8))
fig.suptitle('Normalized GPP DT vs Normalized ET (Seasons)',fontsize=20)
for i,(season,months) in enumerate(seasons.items()):
    ax=axes[i]
    season_data=merged_data[merged_data['Month'].isin(months)]
    GPP_max=season_data['Average_GPP_DT'].max()
    ET_max=season_data['Average_ET'].max()
    season_data['Normalized_GPP']=season_data['Average_GPP_DT']/GPP_max
    season_data['Normalized_ET']=season_data['Average_ET']/ET_max
    for year,color in year_colors.items():
        year_data=season_data[season_data['Year']==year]
        avg_data=year_data.groupby(['Hour','Minute']).mean().reset_index()
        avg_data=avg_data.sort_values(by=['Hour','Minute'])
        ax.plot(avg_data['Normalized_GPP'],avg_data['Normalized_ET'],label=str(year),color=color,lw=2)
        area=np.abs(np.trapz(avg_data['Normalized_ET'],avg_data['Normalized_GPP']))
        areas[season][year]=area
    ax.set_title(season)
    ax.set_xlabel('Normalized GPP',fontsize=16)
    ax.set_ylabel('Normalized ET',fontsize=16)
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.tick_params(axis='both',which='major',labelsize=14)
    ax.legend(title='Year',fontsize=14)
plt.tight_layout(rect=[0,0,1,0.95])
plt.show()
for season in areas:
    for year in areas[season]:
        print(f"Area for {season} season, {year} year: {areas[season][year]}")


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8))
fig.suptitle('ET vs GPP DT (Drought vs Baseline)', fontsize=20)
areas = {}
for i, (season_name, months) in enumerate(seasons.items()):
    ax = axes[i]
    season_data = merged_data[merged_data['Month'].isin(months)]
    drought_data = season_data[season_data['Year'] == drought_year]
    baseline_data = season_data
    baseline_mean = baseline_data.groupby(['Hour', 'Minute']).mean().reset_index()
    for col in ['Average_GPP_DT', 'Average_ET']:
        Q1 = baseline_mean[col].quantile(0.25)
        Q3 = baseline_mean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        baseline_mean = baseline_mean[(baseline_mean[col] >= lower_bound) & (baseline_mean[col] <= upper_bound)]
    drought_data = drought_data.sort_values(by=['Hour', 'Minute'])
    drought_mean = drought_data.groupby(['Hour', 'Minute']).mean().reset_index()
    ax.plot(baseline_mean['Average_ET'], baseline_mean['Average_GPP_DT'], label='Baseline', color='blue', lw=2)
    ax.plot(drought_mean['Average_ET'], drought_mean['Average_GPP_DT'], label='Drought Year', color='red', lw=2)
    baseline_area = np.abs(np.trapz(baseline_mean['Average_GPP_DT'], baseline_mean['Average_ET']))
    drought_area = np.abs(np.trapz(drought_mean['Average_GPP_DT'], drought_mean['Average_ET']))
    areas[season_name] = {'Baseline': baseline_area, 'Drought': drought_area}
    ax.set_title(season_name, fontsize=16)
    ax.set_xlabel("Average ET", fontsize=14)
    ax.set_ylabel("Average GPP", fontsize=14)
    ax.legend(fontsize=12)
    ax.text(0.05, 0.95, f"Baseline Area: {baseline_area:.2f}\nDrought Area: {drought_area:.2f}", transform=ax.transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
for season, values in areas.items():
    print(f"Season: {season}")
    for condition, area in values.items():
        print(f"  {condition} Area: {area:.2f}")


In [ ]:

fig, axes = plt.subplots(2, 3, figsize=(24, 16))
fig.suptitle('Rising and Falling Curves for GPP DT vs ET (Drought vs Baseline)', fontsize=20)
hysteresis_indices = []
for i, (season_name, months) in enumerate(seasons.items()):
    ax_baseline = axes[0, i]
    ax_drought = axes[1, i]
    season_data = merged_data[merged_data['Month'].isin(months)]
    drought_data = season_data[season_data['Year'] == drought_year]
    baseline_data = season_data[season_data['Year'] != drought_year]
    baseline_mean = baseline_data.groupby(['Hour', 'Minute']).mean().reset_index()
    for col in ['Average_GPP_DT', 'Average_ET']:
        Q1 = baseline_mean[col].quantile(0.25)
        Q3 = baseline_mean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        baseline_mean = baseline_mean[(baseline_mean[col] >= lower_bound) & (baseline_mean[col] <= upper_bound)]
    drought_data = drought_data.sort_values(by=['Hour', 'Minute'])
    drought_mean = drought_data.groupby(['Hour', 'Minute']).mean().reset_index()
    baseline_X = baseline_mean['Average_ET'].reset_index(drop=True)
    baseline_Y = baseline_mean['Average_GPP_DT'].reset_index(drop=True)
    drought_X = drought_mean['Average_ET'].reset_index(drop=True)
    drought_Y = drought_mean['Average_GPP_DT'].reset_index(drop=True)
    baseline_max_index = baseline_X.idxmax()
    drought_max_index = drought_X.idxmax()
    rising_baseline_X = baseline_X[:baseline_max_index+1]
    falling_baseline_X = baseline_X[baseline_max_index:]
    rising_baseline_Y = baseline_Y[:baseline_max_index+1]
    falling_baseline_Y = baseline_Y[baseline_max_index:]
    rising_drought_X = drought_X[:drought_max_index+1]
    falling_drought_X = drought_X[drought_max_index:]
    rising_drought_Y = drought_Y[:drought_max_index+1]
    falling_drought_Y = drought_Y[drought_max_index:]
    ax_baseline.plot(rising_baseline_X, rising_baseline_Y, label=f'Rising (Baseline)', color='green', lw=2)
    ax_baseline.plot(falling_baseline_X, falling_baseline_Y, label=f'Falling (Baseline)', color='orange', lw=2)
    ax_drought.plot(rising_drought_X, rising_drought_Y, label=f'Rising (Drought)', color='blue', lw=2)
    ax_drought.plot(falling_drought_X, falling_drought_Y, label=f'Falling (Drought)', color='red', lw=2)
    rising_baseline_area = np.abs(np.trapz(rising_baseline_Y, rising_baseline_X))
    falling_baseline_area = np.abs(np.trapz(falling_baseline_Y, falling_baseline_X))
    rising_drought_area = np.abs(np.trapz(rising_drought_Y, rising_drought_X))
    falling_drought_area = np.abs(np.trapz(falling_drought_Y, falling_drought_X))
    hysteresis_index_baseline = rising_baseline_area / falling_baseline_area
    hysteresis_index_drought = rising_drought_area / falling_drought_area
    hysteresis_indices.append({'Season': season_name, 'Hysteresis_Index_Baseline': hysteresis_index_baseline, 'Hysteresis_Index_Drought': hysteresis_index_drought})
    ax_baseline.set_title(f'Baseline - {season_name}', fontsize=16)
    ax_baseline.set_xlabel("Average ET", fontsize=14)
    ax_baseline.set_ylabel("Average GPP", fontsize=14)
    ax_baseline.legend(fontsize=12)
    ax_drought.set_title(f'Drought - {season_name}', fontsize=16)
    ax_drought.set_xlabel("Average ET", fontsize=14)
    ax_drought.set_ylabel("Average GPP", fontsize=14)
    ax_drought.legend(fontsize=12)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
for index in hysteresis_indices:
    print(f"Season: {index['Season']}, Hysteresis Index (Baseline): {index['Hysteresis_Index_Baseline']}, Hysteresis Index (Drought): {index['Hysteresis_Index_Drought']}")
